In [1]:
import sys 
import pandas as pd
import logging
import numpy as np

sys.path.append('../')

import utils.llm 
import utils.scoring

logger = logging.getLogger()
handler = logging.StreamHandler(sys.stdout)
handler.setLevel(logging.INFO)
logger.addHandler(handler)
logger.setLevel(logging.INFO)



Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [2]:


DATA_DIR = '../data/'
DATA_SAMPLES_PATH = '../data_out/random_query_temperature_0.01_sample_n_30_random_str_length_60_11_23_14_57_54.csv'
NUM_SAMPLES = 30


annotated_df = pd.read_csv(DATA_DIR + "df_JAQKET_qa_annot.csv")
sampled_df = pd.read_csv(DATA_SAMPLES_PATH)




In [3]:
annotated_df = annotated_df[:189]

In [4]:

num_questions = len(annotated_df)


In [5]:
from transformers import logging

logging.set_verbosity_error()

qid = "QA20CAPR-0001"
response = "マンタレイ"
samples = list(sampled_df[sampled_df['qid']  == qid]["sample"])


score_pr, score_re, score_f1 = utils.scoring.get_hallucination_score(response,samples)


In [6]:
score_f1

[0.06792441606521574]

In [7]:
from tqdm import tqdm 

score_df = pd.DataFrame(columns=['query', 'score_pr','score_re','score_f1'])
for i in tqdm(range(num_questions)): 

    # logger.info("processing {} / {}".format(i,num_questions))
    
    qid = annotated_df.iloc[i]['qid']
    response  : str= annotated_df.iloc[i]['generated_answer']
    samples = list(sampled_df[sampled_df['qid'] == qid]["sample"])[:NUM_SAMPLES]
    
    
    pr_scores, re_scores,f1_scores = utils.scoring.get_hallucination_score(response,samples)
    
    
    # score_df = score_df.append({
    #         'qid':qid,
    #         'pr_score':",".join([str(x) for x in pr_scores]), 
    #         're_score':",".join([str(x) for x in re_scores]), 
    #         'f1_score':",".join([str(x) for x in f1_scores]),
    #     },
    #     ignore_index=True)
    
    # use concat instead of append
    
    score_df = pd.concat([
        score_df,
        pd.DataFrame({
            'qid':qid,
            'pr_score':",".join([str(x) for x in pr_scores]), 
            're_score':",".join([str(x) for x in re_scores]), 
            'f1_score':",".join([str(x) for x in f1_scores]),
        }, index=[0])
    ])
    
    



100%|██████████| 189/189 [01:07<00:00,  2.79it/s]


In [8]:
logger.info("done") 

done


In [9]:




result_df = pd.merge(annotated_df,score_df,on='qid')


In [10]:
display(result_df)

,qid,question,answer,generated_answer,factuality,faithfulness,collapsed,Unnamed: 7,備考,query,score_pr,score_re,score_f1,pr_score,re_score,f1_score
0,QA20CAPR-0001,和名をハダカカメガイといい、実は巻き貝の一種とされている、その姿から「流氷の天使」と呼ばれる...,クリオネ,マンタレイ,0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.07975276311238583,0.055734984079996774,0.06792441606521574
1,QA20CAPR-0003,作家のルスティケロが、マルコ・ポーロから聞いた話をまとめた作品といえば何でしょう?,東方見聞録,黄金の島 イタリア ヴェネツィア マルコ・ポーロは、1453年から1547年までヴェネツィア...,"0,0,0,0","1,1,1,1",NaN,NaN,NaN,NaN,NaN,NaN,NaN,"0.23134464025497414,0.1958530465761818,0.18726...","0.2253266632556915,0.2161854922771452,0.191749...","0.23008923927942937,0.21062019268671692,0.1968..."
2,QA20CAPR-0005,『騎士団長殺し』『1Q84』『ノルウェイの森』といった小説の作者は誰でしょう?,村上春樹,村上春樹,1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0949510991573333,0.2285228917996088,0.17947973807652806
3,QA20CAPR-0007,サッカーで、コーナーキックやフリーキックのようにボールを止めた状態から始めるプレーのことを何...,セットプレー,ゴールキック(goal kick)といいます。\n,0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.19833033680915846,0.17578605612119055,0.18746269543965668
4,QA20CAPR-0009,今から約140億年前に起こったとされる、宇宙の始まりの大爆発のことを何というでしょう?,ビッグバン,ビッグバン」といいます。\n,1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.2280647476514176,0.133685143788656,0.19331195354461694
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
184,QA20CAPR-0373,1900年に自動車での旅行を活性化するために始めたレストランのガイドブックが現在は世界的に広...,ミシュラン,ミシュランタイヤです。\n,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.1463006575902311,0.07911650339762424,0.11400649944941245
185,QA20CAPR-0375,1984年に竣工した、大相撲の本場所が行われる東京都墨田区にある施設は何でしょう?,両国国技館,両国国技館です。\n,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.17361146211624123,0.09425753355026195,0.1357522010803217
186,QA20CAPR-0377,プロ野球のシーズンオフに行われる選手の引き抜き合戦のことを、ある暖房器具に引っかけて何という...,ストーブリーグ,ウィンタースポーツ」と言います。\n,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.18894496361414598,0.1868306537469226,0.18878560463587402
187,QA20CAPR-0379,矢澤芸術学院に通う高校生の幸田実果子を主人公とする、矢沢あいの漫画は何でしょう?,ご近所物語,NANAです。\n,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.012107547124226903,0.013305415709813528,0.012773595253626535


In [11]:
result_df.to_csv(DATA_DIR + 'df_JAQKE_selfcheck.csv',index=False)

In [12]:
result_df = pd.read_csv(DATA_DIR + 'df_JAQKE_selfcheck.csv')

def extract_label_and_scores():
    
    df = result_df.copy()    

    df  = df[['qid','factuality','pr_score','re_score','f1_score','question']]
    
    label_and_scores = {
        'label':[],
        'pr':[],
        're':[],
        'f1':[]
    }
    
    # set data type
    df['factuality'] = df['factuality'].astype(str)
    
    for i in range(df.shape[0]):
        labels = df.iloc[i]['factuality'].split(',')
        pr_scores = df.iloc[i]['pr_score'].split(',')
        re_scores = df.iloc[i]['re_score'].split(',')
        f1_scores = df.iloc[i]['f1_score'].split(',')
        
        labels = [1 - int(x) for x in labels]
        pr_scores = [float(x) for x in pr_scores]
        re_scores = [float(x) for x in re_scores]
        f1_scores = [float(x) for x in f1_scores]
        
        assert len(labels) == len(pr_scores), f"labels: {labels}, pr_scores: {pr_scores},i: {i},q: {df.iloc[i]['question']}"
        
        label_and_scores['label'].extend(labels)
        label_and_scores['pr'].extend(pr_scores)
        label_and_scores['re'].extend(re_scores)
        label_and_scores['f1'].extend(f1_scores)
    
    import numpy as np
    label_and_scores["label"] = np.array(label_and_scores["label"])
    label_and_scores["pr"] = np.array(label_and_scores["pr"])
    label_and_scores["re"] = np.array(label_and_scores["re"])
    label_and_scores["f1"] = np.array(label_and_scores["f1"])
    

    return label_and_scores

In [13]:
extract_label_and_scores()

{'label': array([1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0,
        1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0,
        0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0,
        1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0,
        1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0,
        1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0,
        0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1,
        1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1,
        0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1,
        0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1]),
 'pr': array([ 7.97527631e-02,  2.31344640e-01,  1.95853047e-01,  1.87263298e-01,
         2.35025374e-01,  9.49510992e-02,  1.98330337e-01,  2.28064748e-01,
         2.80910975e-01,  1.12726752e-01,  1.58945721e-08,  2.81462351e-02,
         2.57959789e-01,

In [14]:
# AUC scoreを計算   
from sklearn.metrics import average_precision_score
import numpy as np
label_and_scores = extract_label_and_scores()

roc_aucs = {
    'pr':average_precision_score(label_and_scores['label'],label_and_scores['pr']),
    're':average_precision_score(label_and_scores['label'],label_and_scores['re']),
    'f1':average_precision_score(label_and_scores['label'],label_and_scores['f1']),
    "ave":average_precision_score(label_and_scores['label'],(label_and_scores['pr'] + label_and_scores['re'] + label_and_scores['f1'])/3),
    'random' : average_precision_score(label_and_scores['label'],np.random.rand(len(label_and_scores['label']))),
}

print(f"the result (nonfact positive) is {roc_aucs}")

roc_aucs = {
    'pr':average_precision_score(1-label_and_scores['label'],-label_and_scores['pr']),
    're':average_precision_score(1-label_and_scores['label'],-label_and_scores['re']),
    'f1':average_precision_score(1-label_and_scores['label'],-label_and_scores['f1']),
    "ave":average_precision_score(1-label_and_scores['label'],-(label_and_scores['pr'] + label_and_scores['re'] + label_and_scores['f1'])/3),
    'random' : average_precision_score(1-label_and_scores['label'],-np.random.rand(len(label_and_scores['label']))),
}

print(f"the result (fact positive)  is {roc_aucs}")




the result (nonfact positive) is {'pr': 0.6788573121249832, 're': 0.6192898782856913, 'f1': 0.6530023915333922, 'ave': 0.6544747841716742, 'random': 0.5196074536781193}
the result (fact positive)  is {'pr': 0.6243241327915233, 're': 0.6106191091782858, 'f1': 0.6162868042830779, 'ave': 0.616892750650414, 'random': 0.496485043040532}


In [15]:
# PR curveを計算
from sklearn.metrics import precision_recall_curve,roc_auc_score
import matplotlib.pyplot as plt
%matplotlib notebook

label_and_scores = extract_label_and_scores()

pr_curve = precision_recall_curve(label_and_scores['label'],label_and_scores['pr'])
re_curve = precision_recall_curve(label_and_scores['label'],label_and_scores['re'])
f1_curve = precision_recall_curve(label_and_scores['label'],label_and_scores['f1'])

plt.plot(pr_curve[1],pr_curve[0],label='pr')
plt.plot(re_curve[1],re_curve[0],label='re')
plt.plot(f1_curve[1],f1_curve[0],label='f1')
plt.legend()
plt.xlabel('recall')
plt.ylabel('precision')
plt.show()
plt.savefig('pr_curve.png')


<IPython.core.display.Javascript object>

In [16]:
# 閾値を適当に設定して、accuracyを計算

def calc_accuracy(th:float):

    correct_num = {
        'pr':0,
        're':0,
        'f1':0,
    }

    for i in range(len(label_and_scores['label'])):
        label = label_and_scores['label'][i]
        pr = label_and_scores['pr'][i]
        re = label_and_scores['re'][i]
        f1 = label_and_scores['f1'][i]
        
        if pr > th and label == 1:
            correct_num['pr'] += 1
        elif pr <= th and label == 0:
            correct_num['pr'] += 1
            
        if re > th and label == 1:
            correct_num['re'] += 1
        elif re <= th and label == 0:
            correct_num['re'] += 1
            
        if f1 > th and label == 1:
            correct_num['f1'] += 1
        elif f1 <= th and label == 0:
            correct_num['f1'] += 1


    print("acc for th = {}".format(th))
    accs = {
        'pr':correct_num['pr'] / len(label_and_scores['label']),
        're':correct_num['re'] / len(label_and_scores['label']),
        'f1':correct_num['f1'] / len(label_and_scores['label']),
    }

    return accs

import numpy as np
ths = np.linspace(0.08,0.1,10)

for th in ths:
    print(f"th: {th}")
    print(calc_accuracy(th))

th: 0.08
acc for th = 0.08
{'pr': 0.6267281105990783, 're': 0.631336405529954, 'f1': 0.6267281105990783}
th: 0.08222222222222222
acc for th = 0.08222222222222222
{'pr': 0.6267281105990783, 're': 0.6359447004608295, 'f1': 0.6267281105990783}
th: 0.08444444444444445
acc for th = 0.08444444444444445
{'pr': 0.6221198156682027, 're': 0.6405529953917051, 'f1': 0.6267281105990783}
th: 0.08666666666666667
acc for th = 0.08666666666666667
{'pr': 0.6221198156682027, 're': 0.6359447004608295, 'f1': 0.6221198156682027}
th: 0.08888888888888889
acc for th = 0.08888888888888889
{'pr': 0.6267281105990783, 're': 0.6267281105990783, 'f1': 0.6221198156682027}
th: 0.09111111111111111
acc for th = 0.09111111111111111
{'pr': 0.6221198156682027, 're': 0.6267281105990783, 'f1': 0.6221198156682027}
th: 0.09333333333333334
acc for th = 0.09333333333333334
{'pr': 0.6267281105990783, 're': 0.6221198156682027, 'f1': 0.6267281105990783}
th: 0.09555555555555556
acc for th = 0.09555555555555556
{'pr': 0.6313364055299